In [1]:
from numpy.polynomial import Polynomial
from numpy.polynomial.polynomial import polydiv
from phe import paillier
from numpy import prod
import secrets

In [819]:
class Player:
    """
    Class of honest-but-curious adversaries.

    Parameters
    ----------
    S : set_like
    sk : secret key to a homomorpic cryptosystem
    """
    def __init__(self, S, sk):
        self.S = S
        self.k = len(S)
        self.sk = sk
        self.f_list = []
        self.N = sk.public_key.n
    
    def polynomial(self):
        # cria o polinómio f a partir do conjunto S 
        self.f = Polynomial.fromroots(self.S)
    
    def encrypt(self, pk):
        # encipta o polinómio f com pk
        self.ef = self.f #Polynomial([pk.encrypt(x) for x in self.f.coef])                                 HERE
    
    def append_f(self, encrypted_polynomial):
        # adiciona polinomio encriptado de outro jogador a f_list
        self.f_list.append(encrypted_polynomial)

    def receive_l(self, encrypted_polynomial):
        # recebe lambda_i-1
        self.lam_im1 = encrypted_polynomial
    
    def receive_p(self, encrypted_polynomial):
        #recebe p = lambda_n
        self.ep = encrypted_polynomial

    def phi(self):
        # calcula phi_i
        c = len(self.f_list)
        r = []
        for _ in range(c):
            r.append(Polynomial([secrets.randbelow(self.N) for _ in range(self.k + 1)])) 
        self.phi_i = sum([prod([self.f_list[j],r[j]]) for j in range(c)])
        #for x in self.phi_i.coef:                                                                          HERE
        #    x.obfuscate() #re-randomização dos coeficientes

    def lam(self):
        # calcula lambda_i
        self.lam_i = self.lam_im1 + self.phi_i
        #for x in self.lam_i.coef:                                                                          HERE
        #    x.obfuscate() #re-randomização dos coeficientes
    
    def decrypt(self):
        self.p = self.ep #Polynomial([self.sk.decrypt(x) for x in self.ep.coef])                            HERE

    def multiset(self):
        # calcula o multiset de intersecção na forma [(a,b),...,(a,b)], a \in S_i
        # onde o elemento a aparece b vezes no multiset
        res = []
        for elem in self.S:
            degree = 0
            poly = self.p
            quo, rem = polydiv(poly.coef, (-elem, 1))
            while all(rem == 0):
                degree += 1
                poly = quo
                quo, rem = polydiv(poly, (-elem, 1))
            if degree > 0:
                res.append((elem, degree))
        self.intersection = res

    

Soma de polinómios encriptados: `a.ef + b.ef`

Produto de polinímio não encriptado com polinómio encriptado: `a.f * b.ef`

Derivada de polinómio encriptado: `a.ef.deriv(d)`

Avaliação de um polinómio encriptado num ponto não encriptado: `a.ef(x)`


Função da secção 5.1

In [838]:
def psi(players, c, pk):
    """
    Perform a private Set-Intersection secure against a coalition of honest-but-curious
    adversaries.

    Parameters
    ----------
    players : array_like construct of class Player elements
    c : dishonestly colluding constant
    pk: public key to a homomorpic cryptosystem
    """
    n = len(players)
    assert(n >= 2, "Not enough players")
    assert(c < n, "Dishonestly colluding too large")
    assert(all([player.k == players[0].k for player in players]),\
        "Differently sized private sets")
    #1
    for player in players:
        #a
        player.polynomial()       
        player.encrypt(pk)
    
    #b
    for i in range(n):
        for j in range(1, c+1):
            players[i].append_f(players[(i-j) % n].ef)
        
        # Qual é o domínio?? #consoante a resposta é preciso alterar a função phi
        # DEFAULT_KEYSIZE = 3072
        # BASE = 16
        #c, d
        players[i].phi()

    #2
    players[0].lam_i = players[0].phi_i
    players[1].receive_l(players[0].lam_i)

    #3
    for i in range(1, n):
        #a é realizado implicitamente
        #b
        players[i].lam()

        #c
        players[(i+1)%len(players)].receive_l(players[i].lam_i)
    #4
    for player in players:
        player.receive_p(players[0].lam_i)

    #5
    for player in players:
        player.decrypt()
        pass
    #6
    for player in players:
        player.multiset()

<>:13: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:14: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:15: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:13: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:14: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:15: SyntaxWarning: assertion is always true, perhaps remove parentheses?
C:\Users\nefra\AppData\Local\Temp\ipykernel_14040\1799516997.py:13: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(n >= 2, "Not enough players")
C:\Users\nefra\AppData\Local\Temp\ipykernel_14040\1799516997.py:14: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(c < n, "Dishonestly colluding too large")
C:\Users\nefra\AppData\Local\Temp\ipykernel_14040\1799516997.py:15: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(all([player.k == players[0].k for 

In [831]:
# Generate keys
pk, sk = paillier.generate_paillier_keypair(None, 32)
pk.max_int = pk.n

In [832]:
# Create players
a = Player([1,2,3,4,5,6],sk)
b = Player([1,1,1,1,1,1],sk)

In [840]:
# Calculate private set
psi([a,b],1,pk)

In [841]:
#find private set
for x in a.intersection:
    print(x)

dp = a.p.deriv(0)
for x in a.S:
    print(dp(x))

(1, 6)
0.0
325957204550.0
169976347679168.0
9486073918685394.0
1.890044809823068e+17
2.0580142989692188e+18


In [ ]:
#get obtained provate sets

In [ ]:
#compare make sure the sets are the same

In [10]:
# Teste das operações definidas em 4.2
pk, sk = paillier.generate_paillier_keypair()
a = Player([1,2,3],sk)
b = Player([2,3,4],sk)
a.polynomial()
b.polynomial()
a.encrypt(pk)
b.encrypt(pk)

d = 2
x = 0
print(f"Soma:\n{a.ef + b.ef}")
print(f"Multiplicação:\n{a.f * b.ef}")
print(f"Derivada:\n{a.ef.deriv(d)}")
print(f"Avaliação:\n{a.ef(x)}")


Soma:
<phe.paillier.EncryptedNumber object at 0x0000014D6B532290> +
<phe.paillier.EncryptedNumber object at 0x0000014D6BAD5910> x +
<phe.paillier.EncryptedNumber object at 0x0000014D6B75EB10> x**2 +
<phe.paillier.EncryptedNumber object at 0x0000014D6B76CE90> x**3
Multiplicação:
<phe.paillier.EncryptedNumber object at 0x0000014D6B76CE90> +
<phe.paillier.EncryptedNumber object at 0x0000014D6B75EB10> x +
<phe.paillier.EncryptedNumber object at 0x0000014D6B81BB90> x**2 +
<phe.paillier.EncryptedNumber object at 0x0000014D6BAD5910> x**3 +
<phe.paillier.EncryptedNumber object at 0x0000014D6B77A950> x**4 +
<phe.paillier.EncryptedNumber object at 0x0000014D6B819DD0> x**5 +
<phe.paillier.EncryptedNumber object at 0x0000014D6BB9F510> x**6
Derivada:
<phe.paillier.EncryptedNumber object at 0x0000014D6BAD5910> +
<phe.paillier.EncryptedNumber object at 0x0000014D6B76CE90> x
Avaliação:
